In [1]:
!pip3 install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.1 MB/s eta 0:00:00


In [2]:
import pymysql.cursors
import pandas as pd
import datetime
import uuid
import numpy as np
from itertools import combinations


In [4]:

# Connect to the database
db_connection = pymysql.connect(host='159.65.239.201',
                             user='testUser',
                             password='testUser123!',
                             db='sakila',
                             cursorclass=pymysql.cursors.DictCursor)

### Selecting results into a Pandas dataframe

In [5]:

qry = "SELECT * FROM film;"
#print(qry)

result = pd.read_sql(qry, con=db_connection)
result.head()

/var/folders/6d/xqywj2xx2llc_k2t6nkshrgh0000gn/T/ipykernel_29905/2981064506.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(qry, con=db_connection)


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
1,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
2,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
3,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
4,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update


In [5]:
qry = "SELECT * FROM film;"
films = pd.read_sql(qry, con=db_connection)
films.head()

/Users/dbabichenko/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [7]:
qry = "SELECT * FROM actor;"
actor = pd.read_sql(qry, con=db_connection)
actor.head()

/var/folders/6d/xqywj2xx2llc_k2t6nkshrgh0000gn/T/ipykernel_29905/3512026895.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  actor = pd.read_sql(qry, con=db_connection)


,actor_id,first_name,last_name,last_update
0,actor_id,first_name,last_name,last_update
1,actor_id,first_name,last_name,last_update
2,actor_id,first_name,last_name,last_update
3,actor_id,first_name,last_name,last_update
4,actor_id,first_name,last_name,last_update


In [12]:
result = pd.merge(employees, salaries, how='left', left_on='emp_no', right_on='emp_no')

In [13]:
result.shape

(2844047, 9)

### Selecting results using a pymysql cursor

In [9]:

# Connect to the database
con = pymysql.connect(host='159.65.239.201',
                             user='testUser',
                             password='testUser123!',
                             db='sakila',
                             cursorclass=pymysql.cursors.DictCursor)
data = []
try:

    with con.cursor() as cur:

        cur.execute('SELECT * FROM film LIMIT 20;')

        rows = cur.fetchall()

        for row in rows:
            film = {}
            film['film_id'] = row['film_id']
            film['title'] = row['title']
            film['description'] = row['description']
            #print(row['film_id'], row['title'], row['description'])
            data.append(film)

finally:

    con.close()
    
print(data)

[{'film_id': 1, 'title': 'ACADEMY DINOSAUR', 'description': 'A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies'}, {'film_id': 2, 'title': 'ACE GOLDFINGER', 'description': 'A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China'}, {'film_id': 3, 'title': 'ADAPTATION HOLES', 'description': 'A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory'}, {'film_id': 4, 'title': 'AFFAIR PREJUDICE', 'description': 'A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank'}, {'film_id': 5, 'title': 'AFRICAN EGG', 'description': 'A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico'}, {'film_id': 6, 'title': 'AGENT TRUMAN', 'description': 'A Intrepid Panorama of a Robot And a Boy who must Escape a Sumo Wrestler in Ancient China'}, {'film_id': 7, 'title': 'AIRPLANE SI

### Escaping parameters / Parametrized Queries

In [10]:
uname = input("Please enter your user name: ")
password = input("Please enter your password: ")

q = "SELECT * FROM users WHERE uname = '" + uname + "' AND password = '" + password + "'";
print(q)

SELECT * FROM users WHERE uname = 'dbabichenko' AND password = 'test123'


In [12]:
emp_no = input("Please enter employee number: ")
q = "SELECT * FROM employees WHERE emp_no = " + emp_no;
print(q)

SELECT * FROM employees WHERE emp_no = 1; DROP TABLE employees; //


In [ ]:

# Connect to the database
con = pymysql.connect(host='159.65.239.201',
                             user='testUser',
                             password='testUser123!',
                             db='sakila',
                             cursorclass=pymysql.cursors.DictCursor)


film_id = 1 #"1; DELETE FROM film; #"
title = 'ACADEMY DINOSAUR'

#qry = "SELECT * FROM film WHERE "
#qry += "film_id = " + str(film_id) + " AND title ='" + title + "';"

#print(qry)


try: 

    with con.cursor() as cur:
        cur.execute('SELECT * FROM film WHERE film_id=%s AND title=%s', [film_id, title]) 
        
        rows = cur.fetchall()

        for row in rows:
            print(row['film_id'], row['title'], row['description'])
finally:

    con.close()



SELECT * FROM film WHERE film_id = 1; DELETE FROM film; # AND title ='ACADEMY DINOSAUR';


"\ntry: \n\n    with con.cursor() as cur:\n        cur.execute('SELECT * FROM film WHERE film_id=%s AND title=%s', [film_id, title]) \n        \n        rows = cur.fetchall()\n\n        for row in rows:\n            print(row['film_id'], row['title'], row['description'])\nfinally:\n\n    con.close()\n\n"

### Insert a row with PyMySQL

In [28]:
con = pymysql.connect(host='143.198.177.53',
                             user='testUser',
                             password='testUser123@!@!',
                             db='employees',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)



try: 

    with con.cursor() as cur:
        q = "INSERT INTO employees (emp_no, birth_date, first_name, last_name, gender, hire_date) "
        q += "VALUES (%s,%s,%s,%s,%s,NOW());"

        cur.execute(q, (1, '1977-10-10', 'Dmitriy', 'Babichenko', 'M')) 
        con.commit()

finally:

    con.close()


### Update a MySQL table with PyMySQL

In [27]:
con = pymysql.connect(host='143.198.177.53',
                             user='testUser',
                             password='testUser123@!@!',
                             db='employees',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)



try: 

    with con.cursor() as cur:
        qry = 'UPDATE books SET first_name = %s WHERE book_id = %s'

        cur.execute(qry, ('Bob', 3)) 
        con.commit()

finally:

    con.close()


### Delete from a MySQL table with PyMySQL

In [28]:
con = pymysql.connect(host='143.198.177.53',
                             user='testUser',
                             password='testUser123@!@!',
                             db='employees',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)



try: 

    with con.cursor() as cur:
        qry = 'DELETE FROM books WHERE book_id = %s'

        cur.execute(qry, (3)) 
        con.commit()

finally:

    con.close()
